# 주문집행 시뮬레이션
- 크레프트 태크놀로지스 AXE 팀은 주문집행 문제를 강화학습을 통해 해결합니다.
- 하지만 기존의 toy project와는 다르게, 실제 상황에 강화학습을 적용하는 시도는 생각보다 많은 장애물을 마주하게 됩니다.
- 이 샘플 태스크는 그 응용력을 아주 간단한 범위에서 시험합니다.

**주의 : '완벽하게' 푸는 것이 이 태스크의 목적은 아닙니다. 무엇보다, 자신의 생각을 코드로 구현하려는 시도가 중요합니다.**


In [1]:
# 주문집행 시뮬레이션 모듈 import
from env import OrderSimulation1, OrderSimulation2

In [2]:
# 주문 집행 시뮬레이션 인스턴스 - 매수만 고려.
env = OrderSimulation1()

## 함수 스펙 설명

In [3]:
# reset -> 초기 호가정보 반환[5개의 time frame][매수 3호가 ~ 매도 3호가]
order_books = env.reset()
print("호가창 길이: {}".format(len(order_books)))
order_books[-1]

호가창 길이: 5


274000     35
275000    181
276000     61
277000     16
278000     85
279000    121
dtype: int64

In [4]:
# mission_info -> 현재 episode 의 목표 구매 수량과 maximum time step
env.mission_info()

{'total_step': 13, 'mission_buy': 355}

In [5]:
# step -> action(매수 1호가 주문, 매수 2호가 주문, 매수 3호가 주문, 시장가 주문) 을 받아 시뮬레이션을 처리
# 편의상 step 후 체결되지 않은 주문은 모두 취소함을 가정합니다.
# 반환 : 새로운 호가창 시퀀스(1~4번쨰는 이전 시퀀스의 2~5번쨰 항목과 동일), 이번 기 매수체결 정보, 전체 기 매수체결 정보.
# 모든 주문은 기존에 호가창에 있는 대기 잔량이 체결된 후에 체결됩니다.
order_books, executed_this_step, executed_every_step = env.step([30, 10, 20, 5])
executed_this_step

274000    20
275000    10
276000    30
277000     5
dtype: int64

In [6]:
# get_left_observation() 은 일종의 치트 메소드 입니다.
# 현재부터 episode의 끝까지 발생할 다음 호가창 상태를 미리 알려줍니다.
# (이 환경은 상태전이와 action의 독립을 가정하고 있습니다)
env.get_left_observation()

[274000    141
 275000    169
 276000    101
 277000    154
 278000     48
 279000    194
 dtype: int64,
 273000    189
 274000    157
 275000    185
 276000     47
 277000    115
 278000    130
 dtype: int64,
 272000     45
 273000    165
 274000     77
 275000    200
 276000    185
 277000    175
 dtype: int64,
 272000     80
 273000     18
 274000    160
 275000    190
 276000    139
 277000    103
 dtype: int64,
 271000     14
 272000     32
 273000    151
 274000    164
 275000     50
 276000     22
 dtype: int64,
 271000    110
 272000     34
 273000     30
 274000     17
 275000    137
 276000    178
 dtype: int64,
 272000     82
 273000    101
 274000     38
 275000    187
 276000    106
 277000    188
 dtype: int64,
 271000     10
 272000     29
 273000    182
 274000    187
 275000     40
 276000     21
 dtype: int64,
 270000    126
 271000     60
 272000    199
 273000    138
 274000     14
 275000    132
 dtype: int64,
 269000     70
 270000     19
 271000    188
 272000   

# Task - 리워드가 없는데요
강화학습의 핵심은 reward입니다. <br/>
하지만 보시다 시피 주문집행에서 reset과 step은 명시적인 reward를 반환하지 않습니다.(사소하지만 done 역시 마찬가지입니다.) <br/>
기존 OrderSimulation class를 이용, 아래 형식에 맞추어 Gym 환경을 다시 정의해보십시오.<br/>


*HINT* : 이 simulation은 학습을 위한 gym입니다. <br/>
action을 결정하는 데는 사용해서는 안 되지만, reward를 계산하는 데 미래 데이터를 쓰는데 주저하지 마십시오!

*WARNING* : 정답은 없습니다. 그럴 듯한 학습 목표를 세우는 데 초점을 맞추시면 됩니다.

In [ ]:
class OrderGym:
    
    # TODO
    # return: state, reward, done, info
    def reset(self, action):
        raise NotImplementedError

    # TODO
    # return: state, reward, done, info
    def step(self, action):
        raise NotImplementedError

# Task2 - 투자왕 태희
만약 미래의 상태 전이를 알 수 없는 경우(예컨데 온라인 학습 상황)라면 어떻게 학습을 진행할 수 있을까요?

In [8]:
# 이 환경은 미래 상태를 전혀 알 수 없습니다. 실제 동작 역시 step 시 마다 conditionally - stochastic하게 상태가 전이되는 형태입니다.
env = OrderSimulation2()
order_books= env.reset()
try:
    env.get_left_observation()
except Exception as e:
    print(e)

'OrderSimulation2' object has no attribute 'get_left_observation'


# 투자왕 태희
정말 막막한 상황입니다. 당장 학습을 해야 하는데, 시뮬레이션이 끝나기 전까지는 지금의 행동(주문)이 얼마나 효과적인 것이었는지 판단할 기준이 없습니다.<br/>
하지만 불행중 다행히도 자신이 투자의 왕이라고 주장하는 '태희'라는 친구가 옆에서 코칭을 해 주겠다고 합니다! <br/>
다시 강화학습을 시도할 수 있을까요?

In [11]:
# taehee_action -> 자신만의 투자 방식을 가지고 있는 '태희'의 action을 제시합니다.
# '태희'는 믿을만한 투자의 고수입니다.
taehee_action = env.taehee_action()
taehee_action

array([4, 5, 5, 0])

# Reward 정의해내기
'태희'가 우리의 투자에 대한 reward를 제시해주지는 않습니다. 다만 우리가 투자하는 순간에 옆에서 '나라면 이렇게 하겠어' 하고 훈수를 둘 뿐이죠, <br/>
 '태희'의 훈수를 기반으로 Reward를 명시적으로 알 지 못하는 환경의 학습을 설계하십시오(세부 알고리즘은 어떤 것이어도 상관 없습니다)